In [1]:

import numpy as np
import pandas as pd

In [2]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('../dataset/u.user', sep='|', names=u_cols, encoding='latin-1')
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('../dataset/u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../dataset/u.data', sep='\t', names=r_cols, encoding='latin-1')

In [5]:
#timestamp 제거
ratings = ratings.drop('timestamp',axis=1)

In [7]:
#movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id','title']]

In [10]:
#train, test 데이터 분리
from sklearn.model_selection import train_test_split
x =ratings.copy()
y= ratings['user_id']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y)

In [11]:
#정화도(RMSE)를 계산하는 함수
def RMSE(y_true,y_pred):
    return np.sqrt(np.mean((np.array(y_true)-np.array(y_pred))**2))

In [12]:
def score(model):
    id_pairs = zip(x_test['user_id'],x_test['movie_id'])
    y_pred = np.array([model(user,movie)for (user,movie)in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true,y_pred)


In [16]:
# train 데이터로 full matrix구하기
rating_matrix = x_train.pivot(index='user_id',columns='movie_id',values='rating')

In [17]:
# train set의 모든 가능한 아이템 pair의 cosine similarites 계산
from sklearn.metrics.pairwise import cosine_similarity
rating_matrix_t = np.transpose(rating_matrix)
matrix_dummy = rating_matrix_t.copy().fillna(0)
item_similarity = cosine_similarity(matrix_dummy,matrix_dummy)
item_similarity = pd.DataFrame(item_similarity,index = rating_matrix_t.index,columns=rating_matrix_t.index)

In [32]:
# 주어진 영화의 (movie_id) 가중평균 rating을 계산하는 함수,
# 가중치는 주어진 아이템과 다른 아이템 간의 유사도(item__similarity)
def CF_IBCF(user_id, movie_id):
    #현재 영화가 trian set에 있는지 확인
    if movie_id in item_similarity:
        #현재 영화와 다른 영화의 similarity 값 가져오기
        sim_scores = item_similarity[movie_id]
        # 현 사용자의 모든 rating값 가져오기
        user_rating = rating_matrix_t[user_id]
        #사용자가 평가하지 않은 영화 index 가져오기
        non_rating_idx = user_rating[user_rating.isnull()].index
        # 사용자가 평가하지 않은 영화 제거
        user_rating = user_rating.dropna()
        # 사용자가 평가하지 않은 영화의 similarity 값 제거
        sim_scores = sim_scores.drop(non_rating_idx)
        # 현 영화에 대한 예상 rating 계산, 가중치는 현 영화와 사용자가 평가한 영화의 유사도
        mean_rating = np.dot(sim_scores, user_rating)/sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating

In [33]:
score(CF_IBCF)

1.016100198637031